In [ ]:
!pip install basedosdados -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 2.7 MB/s eta 0:00:00


In [ ]:
!pip install epiweeks

In [ ]:
import pandas as pd
import numpy as np
import basedosdados as bd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from epiweeks import Week, Year
import plotly.graph_objects as go
from plotly.subplots import make_subplots

ModuleNotFoundError: No module named 'epiweeks'

In [ ]:
# pegar as estações das capitais
# estacao, codigo, estado
df_estacoes = bd.read_sql("SELECT e.estacao, e.id_estacao, d.sigla_uf, e.latitude, e.longitude  FROM `basedosdados.br_inmet_bdmep.estacao` as e inner join `basedosdados.br_bd_diretorios_brasil.municipio` as d  on e.id_municipio = d.id_municipio WHERE d.capital_uf=1",
                       billing_project_id='cursopython-turma2')

Downloading: 100%|██████████|


In [ ]:
#estacoes que vamos ficar (uma por capital)
estacoes = ['A104','A249','A303','A101','A401','A305','A001','A612','A002','A203','A901','A702','A201','A521','A320','A807','A301','A312','A652','A304','A925','A801','A135','A409','A009','A701']
df_estacoes = df_estacoes[df_estacoes['id_estacao'].isin(estacoes)]
#é importante perceber que não tem florianópolis!

In [ ]:
cidades = bd.read_sql("SELECT nome  FROM `basedosdados.br_bd_diretorios_brasil.municipio` WHERE capital_uf=1",
                       billing_project_id='cursopython-turma2')['nome'].to_list()

Downloading: 100%|██████████|


# Pegando todos os dados para levar para o streamlit

In [ ]:
select = f"""select e.id_municipio, m.id_estacao, m.data,  max(m.temperatura_max) as t_max,
min(m.temperatura_min) as t_min, avg(m.temperatura_max) as t_max_med,
avg(m.temperatura_min) as t_min_med, sum(m.precipitacao_total) as precipitacao,
from `basedosdados.br_inmet_bdmep.microdados` as m
join `basedosdados.br_inmet_bdmep.estacao` as e
on m.id_estacao = e.id_estacao
where m.id_estacao in {estacoes}
and EXTRACT(year from m.data) <2025
and EXTRACT(year from m.data) >2014
group by e.id_municipio, m.id_estacao, m.data"""

select = select.replace('[','(')
select = select.replace(']',')')

df_temps = bd.read_sql(select, billing_project_id='cursopython-turma2')

Downloading: 100%|██████████|


In [ ]:
f"{cidades}"

"['Porto Velho', 'Rio Branco', 'Manaus', 'Boa Vista', 'Belém', 'Macapá', 'Palmas', 'São Luís', 'Teresina', 'Fortaleza', 'Natal', 'João Pessoa', 'Recife', 'Maceió', 'Aracaju', 'Salvador', 'Belo Horizonte', 'Vitória', 'Rio de Janeiro', 'São Paulo', 'Curitiba', 'Florianópolis', 'Porto Alegre', 'Campo Grande', 'Cuiabá', 'Goiânia', 'Brasília']"

In [ ]:
select = f"""select municipio.id_municipio, municipio.nome as cidade,
data_notificacao as data, count(*) as casos
from `basedosdados.br_ms_sinan.microdados_dengue` dengue
join `basedosdados.br_bd_diretorios_brasil.municipio` municipio
on dengue.id_municipio_notificacao = municipio.id_municipio
WHERE classificacao_final IN ("10", "11", "12")
and EXTRACT(year from data_notificacao) <2025
and EXTRACT(year from data_notificacao) >2014
AND tipo_notificacao = "2"
AND municipio.nome in {cidades}
group by municipio.id_municipio, municipio.nome, id_municipio_notificacao, ano, data_notificacao
order by id_municipio_notificacao, ano, data_notificacao
  """

select = select.replace('[','(')
select = select.replace(']',')')

df_casos = bd.read_sql(select, billing_project_id='cursopython-turma2')

Downloading: 100%|██████████|


In [ ]:
df_casos.groupby(by=['id_municipio','data']).casos.count().sort_values()

id_municipio  data      
3304557       2015-09-23    1
              2015-10-03    1
              2015-10-02    1
              2015-10-01    1
              2015-09-30    1
                           ..
2304400       2024-06-09    1
              2024-06-10    1
              2024-06-11    1
              2024-06-12    1
1100205       2022-01-04    2
Name: casos, Length: 68579, dtype: Int64

In [ ]:
df_casos['data']=pd.to_datetime(df_casos['data'])

In [ ]:
df_casos[(df_casos['id_municipio']=='1100205') & (df_casos['data']==pd.to_datetime('2022-01-04'))]

,id_municipio,cidade,data,casos
889,1100205,Porto Velho,2022-01-04,1
891,1100205,Porto Velho,2022-01-04,2


In [ ]:
df_casos.drop(889, inplace=True)

In [ ]:
df_temps['data'] = pd.to_datetime(df_temps['data'])

In [ ]:
df = df_temps.merge(df_casos,left_on=['id_municipio','data'],right_on=['id_municipio','data'],how='left')

In [ ]:
df[['cidade','data','t_max','t_min','t_max_med','t_min_med','precipitacao','casos']].to_csv('diarios.csv')

In [ ]:
df['temperatura_media']=(df['t_max']+df['t_min'])/2

In [ ]:
df['mes'] = df['data'].dt.month
df['ano'] = df['data'].dt.year

In [ ]:
diarios = df[['cidade','data','ano','mes','temperatura_media','precipitacao','casos']]

In [ ]:
diarios.to_csv('diarios.csv',index=False)

In [ ]:
df['semana'] = 0
df['ano'] = 1990

In [ ]:
#essa não é a forma mais eficiente de iterar, mas é a que eu vou usar
for i, row in df.iterrows():
  df.at[i,'semana'] = Week.fromdate(row['data'],'ISO').week
  df.at[i,'ano']=Week.fromdate(row['data'],'ISO').year

In [ ]:
#function_dict = {'t_max':'max','t_min':'min','t_max_med':'mean','t_min_med':'mean','precipitacao':'sum'}
function_dict = {'temperatura_media':'mean','precipitacao':'sum','casos':'sum'}
semanal = df.groupby(by=['id_municipio','cidade','ano','semana']).aggregate(function_dict).reset_index()


In [ ]:
df_temps[df_temps['id_municipio']=='1100205']['t_max'].isna().count()

np.int64(3622)

In [ ]:
semanal.to_csv('semanais.csv',index=False)